In [1]:
from pathlib import Path
import pandas as pd
import sqlite3
from src.utils.os_helper import get_project_root
import os

In [2]:
repo = get_project_root()

CACHE_DIR = repo / "cache" / "data" / "awesome_cgm" / "aleppo"
data_tables = CACHE_DIR / "raw" / "Data Tables"
db_path = CACHE_DIR / "awesome_cgm.db"

# SQLite param cap (commonly 999). Keep a margin.
SQLITE_MAX_VARS = 999
MARGIN = 10

In [ ]:
con = sqlite3.connect(db_path)
cur = con.cursor()
cur.execute("PRAGMA journal_mode=WAL;")
cur.execute("PRAGMA synchronous=OFF;")
cur.execute("PRAGMA temp_store=MEMORY;")

for f in sorted(data_tables.glob("*.txt")):
    table = f.stem
    print(f"Importing {f.name} -> {table}")

    # Stream in chunks to control memory and SQL variables
    first = True
    for df in pd.read_csv(f, sep="|", dtype=str, low_memory=False, chunksize=50_000):
        ncols = len(df.columns)
        # rows per insert so (rows * cols) <= SQLITE_MAX_VARS - MARGIN
        safe_rows = max(1, (SQLITE_MAX_VARS - MARGIN) // max(1, ncols))

        # method=None avoids multi-row SQL text construction; Pandas will executemany
        df.to_sql(
            table,
            con,
            if_exists="replace" if first else "append",
            index=False,
            chunksize=safe_rows,
            method=None,
        )
        first = False


indexes = [
    # Bolus Indexes
    "CREATE INDEX idx_hdevicebolus_ptid ON HDeviceBolus(PtID);",
    "CREATE INDEX idx_hdevicebolus_parentid ON HDeviceBolus(ParentHDeviceUploadsID);",
    "CREATE INDEX idx_hdevicebolus_ptid_days ON HDeviceBolus(PtID, DeviceDtTmDaysFromEnroll);",
    "CREATE INDEX idx_hdevicebolus_normal ON HDeviceBolus(Normal) WHERE Normal IS NOT NULL;",
    "CREATE INDEX idx_hdevicebolus_order ON HDeviceBolus(PtID, DeviceDtTmDaysFromEnroll, DeviceTm);",
    # Uploads Indexes
    "CREATE INDEX idx_hdeviceuploads_recid ON HDeviceUploads(RecID);",
    "CREATE INDEX idx_hdeviceuploads_ptid ON HDeviceUploads(PtID);",
    "CREATE INDEX idx_hdeviceuploads_device ON HDeviceUploads(DeviceModel, DeviceType);",
    # CGM Indexes
    "CREATE INDEX idx_hdevicecgm_recid ON HDeviceCGM(RecID);",
    "CREATE INDEX idx_hdevicecgm_recordtype ON HDeviceCGM(RecordType);",
    "CREATE INDEX idx_hdevicecgm_ptid_days ON HDeviceCGM(PtID, DeviceDtTmDaysFromEnroll);",
    "CREATE INDEX idx_hdevicecgm_ptid_time ON HDeviceCGM(PtID, DeviceTm);",
    # Wizard Indexes
    "CREATE INDEX idx_hdevicewizard_recid ON HDeviceWizard(RecID);",
    "CREATE INDEX idx_hdevicewizard_ptid ON HDeviceWizard(PtID);",
    "CREATE INDEX idx_hdevicewizard_ptid_days ON HDeviceWizard(PtID, DeviceDtTmDaysFromEnroll);",
    "CREATE INDEX idx_hdevicewizard_ptid_time ON HDeviceWizard(PtID, DeviceTm);",
]

for i, index_sql in enumerate(indexes):
    try:
        cur.execute(index_sql)
        print(
            f"Created index {i+1}/{len(indexes)}: {index_sql.split('ON ')[1].split('(')[0]}"
        )
    except sqlite3.Error as e:
        print(f"Error creating index: {e}")

con.commit()
con.close()

In [ ]:
def query_db(query: str):
    # Open the connection first
    con = sqlite3.connect(db_path)
    # cur = con.cursor()
    df = pd.read_sql_query(query, con)
    con.close()
    return df

### Convert data to csv

- pid
- date
- tableType
- bolusType
- normalBolus
- expectedNormalBolus
- extendedBolus
- expectedExtendedBolus
- bgInput
- foodG
- iob
- cr
- isf
- bgMgdl
- rate
- suprBasalType
- suprRate

In [ ]:
query = """
WITH
  params AS (
    SELECT '2020-01-01' AS base_date
  )
SELECT
    pid, date, tableType,
    bolusType, normalBolus, expectedNormalBolus, extendedBolus, expectedExtendedBolus,
    bgInput, foodG, iob, cr, isf,
    bgMgdl,
    rate, suprBasalType, suprRate
FROM (
    -- Bolus data
    SELECT
        HDeviceBolus.PtID as pid,
        datetime(julianday((SELECT base_date FROM params)) + CAST(HDeviceBolus.DeviceDtTmDaysFromEnroll AS INTEGER) + (julianday(HDeviceBolus.DeviceTm) - julianday('00:00:00'))) AS date,
        'bolus' as tableType,
        HDeviceBolus.BolusType as bolusType,
        Normal as normalBolus,
        ExpectedNormal as expectedNormalBolus,
        Extended as extendedBolus,
        ExpectedExtended as expectedExtendedBolus,
        NULL as bgInput,
        NULL as foodG,
        NULL as iob,
        NULL as cr,
        NULL as isf,
        NULL as bgMgdl,
        NULL as rate,
        NULL as suprBasalType,
        NULL as suprRate
    FROM HDeviceBolus

    UNION ALL

    -- Wizard data
    SELECT
        HDeviceWizard.PtID as pid,
        datetime(julianday((SELECT base_date FROM params)) + CAST(HDeviceWizard.DeviceDtTmDaysFromEnroll AS INTEGER) + (julianday(HDeviceWizard.DeviceTm) - julianday('00:00:00'))) AS date,
        'wizard' as tableType,
        NULL as bolusType,
        NULL as normalBolus,
        NULL as expectedNormalBolus,
        NULL as extendedBolus,
        NULL as expectedExtendedBolus,
        HDeviceWizard.BgInput as bgInput,
        HDeviceWizard.CarbInput / 1000 as foodG,
        HDeviceWizard.InsulinOnBoard as iob,
        HDeviceWizard.InsulinCarbRatio as cr,
        HDeviceWizard.InsulinSensitivity as isf,
        NULL as bgMgdl,
        NULL as rate,
        NULL as suprBasalType,
        NULL as suprRate
    FROM HDeviceWizard

    UNION ALL

    -- CGM data
    SELECT
        HDeviceCGM.PtID as pid,
        datetime(julianday((SELECT base_date FROM params)) + CAST(HDeviceCGM.DeviceDtTmDaysFromEnroll AS INTEGER) + (julianday(HDeviceCGM.DeviceTm) - julianday('00:00:00'))) AS date,
        'cgm' as tableType,
        NULL as bolusType,
        NULL as normalBolus,
        NULL as expectedNormalBolus,
        NULL as extendedBolus,
        NULL as expectedExtendedBolus,
        NULL as bgInput,
        NULL as foodG,
        NULL as iob,
        NULL as cr,
        NULL as isf,
        HDeviceCGM.GlucoseValue as bgMgdl,
        NULL as rate,
        NULL as suprBasalType,
        NULL as suprRate
    FROM HDeviceCGM
    WHERE HDeviceCGM.GlucoseValue IS NOT NULL
    AND HDeviceCGM.RecordType = 'CGM'

    UNION ALL

    -- Basal data
    SELECT
        HDeviceBasal.PtID as pid,
        datetime(julianday((SELECT base_date FROM params)) + CAST(HDeviceBasal.DeviceDtTmDaysFromEnroll AS INTEGER) + (julianday(HDeviceBasal.DeviceTm) - julianday('00:00:00'))) AS date,
        'basal' as tableType,
        NULL as bolusType,
        NULL as normalBolus,
        NULL as expectedNormalBolus,
        NULL as extendedBolus,
        NULL as expectedExtendedBolus,
        NULL as bgInput,
        NULL as foodG,
        NULL as iob,
        NULL as cr,
        NULL as isf,
        NULL as bgMgdl,
        HDeviceBasal.Rate as rate,
        HDeviceBasal.SuprBasalType as suprBasalType,
        HDeviceBasal.SuprRate as suprRate
    FROM HDeviceBasal
)
ORDER BY pid, date ASC;
"""
df_all = query_db(query)
df_all.to_csv("output.csv")

### Raw -> Interim

In [9]:
def convert_to_csv(df):
    project_root = get_project_root()
    data_dir = project_root / "cache" / "data" / "awesome_cgm" / "aleppo" / "interim"
    os.makedirs(data_dir, exist_ok=True)
    for pid in df["pid"].unique():
        df_pid = df[df["pid"] == pid]
        df_pid.to_csv(data_dir / f"p{pid}_full.csv", index=False)
        print(f"Done processing pid {pid}")


convert_to_csv(df_all)

Done processing pid 10
Done processing pid 101
Done processing pid 102
Done processing pid 103
Done processing pid 105
Done processing pid 106
Done processing pid 108
Done processing pid 109
Done processing pid 11
Done processing pid 110
Done processing pid 111
Done processing pid 112
Done processing pid 113
Done processing pid 115
Done processing pid 116
Done processing pid 118
Done processing pid 119
Done processing pid 121
Done processing pid 123
Done processing pid 124
Done processing pid 127
Done processing pid 128
Done processing pid 129
Done processing pid 130
Done processing pid 131
Done processing pid 132
Done processing pid 134
Done processing pid 135
Done processing pid 136
Done processing pid 137
Done processing pid 138
Done processing pid 139
Done processing pid 14
Done processing pid 140
Done processing pid 141
Done processing pid 143
Done processing pid 145
Done processing pid 146
Done processing pid 147
Done processing pid 148
Done processing pid 149
Done processing pid

In [3]:
from src.data.models import ColumnNames
from src.data.preprocessing.pipeline import preprocessing_pipeline
from src.utils.unit import mg_dl_to_mmol_l
import pandas as pd
from src.utils.os_helper import get_project_root

import logging

logger = logging.getLogger(__name__)


# 	pid	date	tableType	bolusType	normalBolus	expectedNormalBolus	extendedBolus	expectedExtendedBolus	bgInput	foodG	iob	cr	isf	bgMgdl	rate	suprBasalType	suprRate
def data_translation(df_raw: pd.DataFrame) -> pd.DataFrame:
    """
    'pid' -> p_num
    'date' -> datetime
    'tableType' -> msg_type (bolus, wizard, cgm). wizard contains information like carbs intake
    'eventType': This is bolus type
    'normal': Number of units of normal bolus
    'expectedNormal'
    'extended': Number of units for extended delivery
    'expectedExtended'
    'bgInput' -> Blood glucose as inputted into wizard in mg
    'carbInput' -> Carbohydrates as inputted into wizard in mg
    'iob': Units of insulin on board
    'cr': Number of mg carbs covered by unit of insulin. Not used yet but we can find a way to give model a hint about the slope of the glucose curve
    'isf': Number of bgs covered by unit of insulin. Same as above
    'recordType': CGM | CALIBRATION
    'glucoseValue' -> bg_mM
    """
    df = df_raw.copy()
    # TODO: Rename to the correct column names
    df = df.rename(
        columns={
            "pid": ColumnNames.P_NUM.value,
            "date": ColumnNames.DATETIME.value,
            "tableType": ColumnNames.MSG_TYPE.value,
            "normalBolus": ColumnNames.DOSE_UNITS.value,
            # "expectedNormalBolus": ColumnNames.EXPECTED_NORMAL.value,
            # "extendedBolus": ColumnNames.EXTENDED.value,
            # "expectedExtendedBolus": ColumnNames.EXPECTED_EXTENDED.value,
            # "bgInput": ColumnNames.BG.value, todo: Maybe tag the record type as bgInput
            "foodG": ColumnNames.FOOD_G.value,
            "iob": ColumnNames.IOB.value,
            # "cr": ColumnNames.INSULIN_CARB_RATIO.value,
            # "isf": ColumnNames.ISF.value,
            "recordType": ColumnNames.RECORD_TYPE.value,
            "bgMgdl": ColumnNames.BG.value,
            "rate": ColumnNames.RATE.value,
            "suprBasalType": ColumnNames.SUPR_BASAL_TYPE.value,
            "suprRate": ColumnNames.SUPR_RATE.value,
        }
    )
    df.drop(columns=["expectedNormalBolus", "expectedExtendedBolus"], inplace=True)
    df.set_index(ColumnNames.DATETIME.value, inplace=True)
    df.sort_index(inplace=True)
    df.index = pd.to_datetime(df.index)

    # Convert blood glucose from mg/dL to mmol/L
    df[ColumnNames.BG.value] = mg_dl_to_mmol_l(df, bgl_col=ColumnNames.BG.value)

    return df


def keep_overlapping_data(patient_df: pd.DataFrame) -> pd.DataFrame:
    """
    Args:
        patient_df: A dataframe that has been through data_translation (datetime is the index)

    Keep data that has overlapping time windows for all table types.
    Note that not all patients have data for all table types so we only consider the table types that are present.

    Don't think we are losing too much data here. Most patients still have at least 6 months worth of data after this step except for some patients.
    - p216, p019, p081, p289, p138 (check 3.18 notebook for more details)
    """
    table_types = patient_df[ColumnNames.MSG_TYPE.value].unique()
    start_datetime = None  # This should be the max of all the min datetimes
    end_datetime = None  # This should be the min of all the max datetimes

    for table_type in table_types:
        table_data = patient_df[patient_df[ColumnNames.MSG_TYPE.value] == table_type]
        if table_data.empty:
            continue

        min_datetime = table_data.index.min()
        max_datetime = table_data.index.max()

        # Find the latest start time of all table types
        if start_datetime is None or min_datetime > start_datetime:
            start_datetime = min_datetime

        # Find the earliest end time of all table types
        if end_datetime is None or max_datetime < end_datetime:
            end_datetime = max_datetime

    if start_datetime is None or end_datetime is None:
        return None

    has_overlap = start_datetime < end_datetime
    if not has_overlap:
        # This shouldn't happen
        logger.warning(
            f"Patient {patient_df[ColumnNames.P_NUM.value].iloc[0]} has no overlapping data"
        )
        return None

    # Filter using the index (datetime)
    return patient_df[
        (patient_df.index >= start_datetime) & (patient_df.index <= end_datetime)
    ]

In [4]:
from src.data.preprocessing.generic_cleaning import erase_consecutive_nan_values
from src.data.preprocessing.time_processing import get_most_common_time_interval


def process_one_patient(
    df_raw: pd.DataFrame, debug: bool = False, verbose: bool = False
) -> pd.DataFrame:
    """
    Process the raw data for one patient:
        1. Translate the data (columns and units)
        2. Keep overlapping data (for bolus, wizard, cgm and basal)
        3. Rollover basal rate to the next few rows if the rate is not null
        4. preprocessing_pipeline (This include resampling and deriving cob and iob)
    """
    HOURS_OF_CONSECUTIVE_NAN_VALUES = 6
    df = df_raw.copy()

    # Translate the data
    df = data_translation(df)
    pid = df[ColumnNames.P_NUM.value].iloc[0]

    # Keep overlapping data
    logger.info(f"Keeping overlapping data for patient {pid}")
    df = keep_overlapping_data(df)
    if df is None:
        return None

    # Print data meta: span of datetime index and number of rows
    if verbose:
        start_dt = df.index.min()
        end_dt = df.index.max()
        food_g = df[df[ColumnNames.FOOD_G.value].notna()]
        bolus = df[df[ColumnNames.DOSE_UNITS.value].notna()]
        logger.info(
            f"Patient {pid} processed data spans from {start_dt} to {end_dt} ({(end_dt - start_dt)})"
        )
        logger.info(f"Number of rows with food intake: {len(food_g)}")
        logger.info(f"Number of rows with bolus: {len(bolus)}")

    # Resampling to constant interval, rollover basal rate and derive cob and iob
    df = preprocessing_pipeline(pid, df, use_aggregation=True)

    # Drop days with more than 6 hours of consecutive NaN values
    freq_mins = get_most_common_time_interval(df)
    max_consecutive_nan_values_per_day = (
        HOURS_OF_CONSECUTIVE_NAN_VALUES * 60
    ) // freq_mins
    df = erase_consecutive_nan_values(
        df, max_consecutive_nan_values_per_day=max_consecutive_nan_values_per_day
    )

    # Debug only
    if debug:
        debug_dir = (
            get_project_root() / "cache" / "data" / "awesome_cgm" / "aleppo" / "debug"
        )
        os.makedirs(debug_dir, exist_ok=True)
        df.to_csv(debug_dir / f"p{pid}.csv", index=True)
    return df


def process_all_patients(
    interim_path: Path,
    processed_path: Path,
) -> pd.DataFrame:
    """
    Clean all patients' data in the interim path and save the processed data to the processed path.
    """
    for pid in os.listdir(interim_path):
        df = pd.read_csv(interim_path / pid)
        df = process_one_patient(df)
        df.to_csv(processed_path / pid, index=True)
        print(f"{"-"*10}Done processing pid {pid} {"-"*10}")
    return

### Interim -> Processed for one patient

In [ ]:
PID = 7

root = get_project_root()
patient_df = pd.read_csv(
    f"{root}/cache/data/awesome_cgm/aleppo/interim/p{PID}_full.csv"
)

processed_df = process_one_patient(patient_df)
processed_df.to_csv("p{PID}_full.csv", index=True)

2025-10-28T03:28:24 - Keeping overlapping data for patient 7
2025-10-28T03:28:24 - ==============================
2025-10-28T03:28:24 - Preprocessing patient 7
2025-10-28T03:28:24 - ==============================
2025-10-28T03:28:24 - create_physiological_features(): Deriving features...
2025-10-28T03:28:24 - 	Ensuring regular time intervals with aggregation...
2025-10-28T03:28:24 - ensure_regular_time_intervals_with_aggregation(): Ensuring regular time intervals with aggregation...
2025-10-28T03:28:24 - 	Most common time interval: 5 minutes
2025-10-28T03:28:24 - 	Aggregation strategy: {'p_num': <function ensure_regular_time_intervals_with_aggregation.<locals>.<lambda> at 0x1669cdf80>, 'msg_type': <function ensure_regular_time_intervals_with_aggregation.<locals>.<lambda> at 0x1669ce020>, 'bolusType': <function ensure_regular_time_intervals_with_aggregation.<locals>.<lambda> at 0x1669cde40>, 'dose_units': 'sum', 'extendedBolus': 'sum', 'bgInput': 'sum', 'food_g': 'sum', 'iob': <function

In [7]:
processed_df.to_csv(f"p{PID}_full.csv", index=True)

In [13]:
cgm_df = patient_df[patient_df["tableType"] == "cgm"]
cgm_start = cgm_df.date.min()
cgm_end = cgm_df.date.max()
print(f"CGM data spans from {cgm_start} to {cgm_end}")


bolus_df = patient_df[patient_df["tableType"] == "bolus"]
bolus_start = bolus_df.date.min()
bolus_end = bolus_df.date.max()
print(f"Bolus data spans from {bolus_start} to {bolus_end}")

wizard_df = patient_df[patient_df["tableType"] == "wizard"]
wizard_start = wizard_df.date.min()
wizard_end = wizard_df.date.max()
print(f"Wizard data spans from {wizard_start} to {wizard_end}")

basal_df = patient_df[patient_df["tableType"] == "basal"]
basal_start = basal_df.date.min()
basal_end = basal_df.date.max()
print(f"Basal data spans from {basal_start} to {basal_end}")


data_start = max(cgm_start, bolus_start, wizard_start, basal_start)
data_end = min(cgm_end, bolus_end, wizard_end, basal_end)
print(f"Data spans from {data_start} to {data_end}")


cgm_df.to_csv("cgm_p7.csv")

CGM data spans from 2019-08-17 14:32:00 to 2020-09-30 10:01:44
Bolus data spans from 2019-11-01 20:38:27 to 2020-09-30 08:58:17
Wizard data spans from 2019-11-02 09:19:27 to 2020-09-30 08:58:17
Basal data spans from 2019-11-01 23:12:23 to 2020-09-30 10:14:30
Data spans from 2019-11-02 09:19:27 to 2020-09-30 08:58:17


### Process all patients

In [9]:
interim_path = repo / "cache" / "data" / "awesome_cgm" / "aleppo" / "interim"
processed_path = repo / "cache" / "data" / "awesome_cgm" / "aleppo" / "processed"

process_all_patients(interim_path, processed_path)

2025-10-28T04:10:07 - Keeping overlapping data for patient 47
2025-10-28T04:10:07 - ==============================
2025-10-28T04:10:07 - Preprocessing patient 47
2025-10-28T04:10:07 - ==============================
2025-10-28T04:10:07 - create_physiological_features(): Deriving features...
2025-10-28T04:10:07 - 	Ensuring regular time intervals with aggregation...
2025-10-28T04:10:07 - ensure_regular_time_intervals_with_aggregation(): Ensuring regular time intervals with aggregation...
2025-10-28T04:10:07 - 	Most common time interval: 5 minutes
2025-10-28T04:10:07 - 	Aggregation strategy: {'p_num': <function ensure_regular_time_intervals_with_aggregation.<locals>.<lambda> at 0x1773cdbc0>, 'msg_type': <function ensure_regular_time_intervals_with_aggregation.<locals>.<lambda> at 0x1773cdda0>, 'bolusType': <function ensure_regular_time_intervals_with_aggregation.<locals>.<lambda> at 0x1773cd8a0>, 'dose_units': 'sum', 'extendedBolus': 'sum', 'bgInput': 'sum', 'food_g': 'sum', 'iob': <functi

----------Done processing pid p47_full.csv ----------


2025-10-28T04:12:48 - Post-ensure_regular_time_intervals_with_aggregation(): 
			Patient 46 
			 - old index length: 64308, 
			 - new index length: 67378
2025-10-28T04:12:48 - 	Rollover basal rate...
/Users/tonychan/GlucoseML/nocturnal-hypo-gly-prob-forecast/src/data/preprocessing/feature_engineering.py:117: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the doc

----------Done processing pid p46_full.csv ----------


2025-10-28T04:14:31 - Post-ensure_regular_time_intervals_with_aggregation(): 
			Patient 260 
			 - old index length: 35763, 
			 - new index length: 42163
2025-10-28T04:14:31 - 	Rollover basal rate...
2025-10-28T04:14:31 - 	Creating COB/IOB and availability columns. This may take a while depending on the size of the data.
2025-10-28T04:14:31 - 	Creating COB and carb availability columns...
2025-10-28T04:14:31 - 	Processing glucose dynamics
2025-10-28T04:15:01 - 	Creating IOB and insulin availability columns...
2025-10-28T04:15:01 - 	Processing insulin dynamics
2025-10-28T04:15:02 - 	Done deriving features.

2025-10-28T04:15:02 - Keeping overlapping data for patient 147
2025-10-28T04:15:02 - ==============================
2025-10-28T04:15:02 - Preprocessing patient 147
2025-10-28T04:15:02 - ==============================
2025-10-28T04:15:02 - create_physiological_features(): Deriving features...
2025-10-28T04:15:02 - 	Ensuring regular time intervals with aggregation...
2025-10-28T04:15

...

----------Done processing pid p22_full.csv ----------


2025-10-28T16:28:23 - Post-ensure_regular_time_intervals_with_aggregation(): 
			Patient 2 
			 - old index length: 68860, 
			 - new index length: 68700
2025-10-28T16:28:24 - 	Rollover basal rate...
/Users/tonychan/GlucoseML/nocturnal-hypo-gly-prob-forecast/src/data/preprocessing/feature_engineering.py:117: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the docu

----------Done processing pid p2_full.csv ----------


2025-10-28T16:30:28 - Post-ensure_regular_time_intervals_with_aggregation(): 
			Patient 123 
			 - old index length: 78033, 
			 - new index length: 74002
2025-10-28T16:30:28 - 	Rollover basal rate...
/Users/tonychan/GlucoseML/nocturnal-hypo-gly-prob-forecast/src/data/preprocessing/feature_engineering.py:117: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the do

----------Done processing pid p123_full.csv ----------


### TODO

- [x] Figure out why we are missing some patients like p04 - There is no reason
- [x] Verify db is created correctly in the right directory
- [x] Figure out the columns of the tables and make sure the unit is actually correct. Is CarbInput in g and "normal" in units at all?
- [ ] Plot a histogram to see if there are significantly less data before the enrollment day.
- [ ] Figure out how to handle missing data (Some patients have missing cgm data). - Need to filter out calibaration. That seems to remove some of the large gap
- [ ] Union with Basal table too (the rate should be roll to the next few rows. For example, a patient with 5-min interval has a rate of 0.1 units/hour, then we should have a 12 rows of 0.1/12 unit each). Also need to do a addition with whatever bolus is already there. So if a row already has 8 units of insulin, then it will be 8 + 0.1/12 units. Also need to keep rate as 0 if null.
- [ ] Take a look at some patients' raw data and see if it makes sense at all first


## Loader
- [ ] Add a test split for the loader (5% of the entire dataset is enough)
- [ ] Verify the `clean_all_patients` is working
- [ ] Verify that `data_loader` works at all
- [ ] For the loader class, cross check with Kaggle to do a similar data check
- [ ] Update `README.md`
